In [4]:
# import libraries
#import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import gzip
import pickle
import os
from urllib.request import urlretrieve
import numpy as np
from matplotlib import image


def conv(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_depth = f.shape[1] #f1 = 1 , f2 = 8
    f_size = f.shape[2] 
        

    h_range = int((image.shape[1] - f_size) / s) + 1     # (W - F + 2P) / S  
    w_range = int((image.shape[2] - f_size) / s) + 1     # (W - F + 2P) / S  
    np_o = np.zeros((f_num, h_range, w_range))
    for z in range(f_num): # Number of filters
        for _h in range(h_range):      
            for _w in range(w_range):
                np_o[z, _h, _w] = np.sum(np.multiply(image[:, _h*s : _h*s + f_size, _w*s : _w*s + f_size] , f[z, :, :, :])) + b[z]
    
    return np_o
def conv_alg(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_size = f.shape[2] 
    
    np_o_t = np.zeros((f_num, image.shape[1] + (f_size-1), image.shape[2]+ (f_size-1)))
    for z in range(f_num): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                np_o_t[z,_h:(_h+f_size),_w:(_w+f_size)] += np.sum((image[:, _h, _w].reshape(-1,1,1)*f[z, :, :, :]),axis=0)
        np_o_t[z,:,:] += b[z]*np.ones((image.shape[1] + (f_size-1),image.shape[2] + (f_size-1)))
    return np_o_t[:,(f_size-1):-(f_size-1), (f_size-1):-(f_size-1)]

def convTransp(image, params, s = 1, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f = np.rot90(f, 2, (2,3))
    params = [f, b]
    input_s = image.shape[1]
    target_dim = input_s*2 #final dim, after conv
    required_pad = 1 #always for filter 2x2 ,stride 1, zero insertion 1 and main target to double dim
    #make our new custom input
    size = input_s*2 +1
    new_in = np.zeros((image.shape[0], size, size))
    
    ### OR just: np.pad(image[:,:,:],2,'constant') # Important, we must loop with respect to the 1st dim
    for i in range(1, target_dim, 2):
        for j in range(1, target_dim, 2):
                new_in[:, i, j] = image[:, i//2, j//2]
    #now we do a normal convolution(pad = 0)
    res = conv(new_in, params, 1, 0) #thats the final result with target_dim
    return res, new_in # we will need new_in so we wont calc it again for the backprop
def convTransp_alg(image, params, s = 2, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f_size = f.shape[2]
    res = np.zeros((f.shape[0], 2*image.shape[1], 2*image.shape[2]))
    for z in range(f.shape[0]): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                res[z, _h*s : _h*s + f_size, _w*s : _w*s + f_size] += np.sum((image[:, _h, _w].reshape(-1,1,1)) * f[z], axis=0)
        res[z,:,:] += b[z]*np.ones((image.shape[1]*2,image.shape[2]*2))
    return res # we will need new_in so we wont calc it again for the backprop

In [12]:
x = np.ones((2,8,8))

for c in range(2):
    for i in range(8):
        for j in range(8):
            x[c,i,j] = (i+1)*(j*2+1)*1.3 + c
print(x)
#k = np.random.rand(1,2,3,3)
k=np.ones((2,2,3,3))

counter = 1
for n in range(2):
    for c in range(2):
        for i in range(3):
            for j in range(3):
                k[n,c,i,j] = counter
                counter=counter+1

print(k)


b=np.zeros((2))

[[[  1.3   3.9   6.5   9.1  11.7  14.3  16.9  19.5]
  [  2.6   7.8  13.   18.2  23.4  28.6  33.8  39. ]
  [  3.9  11.7  19.5  27.3  35.1  42.9  50.7  58.5]
  [  5.2  15.6  26.   36.4  46.8  57.2  67.6  78. ]
  [  6.5  19.5  32.5  45.5  58.5  71.5  84.5  97.5]
  [  7.8  23.4  39.   54.6  70.2  85.8 101.4 117. ]
  [  9.1  27.3  45.5  63.7  81.9 100.1 118.3 136.5]
  [ 10.4  31.2  52.   72.8  93.6 114.4 135.2 156. ]]

 [[  2.3   4.9   7.5  10.1  12.7  15.3  17.9  20.5]
  [  3.6   8.8  14.   19.2  24.4  29.6  34.8  40. ]
  [  4.9  12.7  20.5  28.3  36.1  43.9  51.7  59.5]
  [  6.2  16.6  27.   37.4  47.8  58.2  68.6  79. ]
  [  7.5  20.5  33.5  46.5  59.5  72.5  85.5  98.5]
  [  8.8  24.4  40.   55.6  71.2  86.8 102.4 118. ]
  [ 10.1  28.3  46.5  64.7  82.9 101.1 119.3 137.5]
  [ 11.4  32.2  53.   73.8  94.6 115.4 136.2 157. ]]]
[[[[ 1.  2.  3.]
   [ 4.  5.  6.]
   [ 7.  8.  9.]]

  [[10. 11. 12.]
   [13. 14. 15.]
   [16. 17. 18.]]]


 [[[19. 20. 21.]
   [22. 23. 24.]
   [25. 26. 27.]]

  [

In [13]:
params = [k,b]
#res_ref,_ = convTransp(x,params)#padding = 0
#res = convTransp_alg(x,params)
#print(res_ref)
res = conv(x,params,1,1)
print(res)

[[[  446.2   931.5  1469.7  2007.9  2546.1  3084.3  3622.5  2470.2]
  [  789.   1662.6  2645.4  3628.2  4611.   5593.8  6576.6  4464.6]
  [ 1108.8  2360.7  3788.1  5215.5  6642.9  8070.3  9497.7  6438. ]
  [ 1428.6  3058.8  4930.8  6802.8  8674.8 10546.8 12418.8  8411.4]
  [ 1748.4  3756.9  6073.5  8390.1 10706.7 13023.3 15339.9 10384.8]
  [ 2068.2  4455.   7216.2  9977.4 12738.6 15499.8 18261.  12358.2]
  [ 2388.   5153.1  8358.9 11564.7 14770.5 17976.3 21182.1 14331.6]
  [ 1432.6  3074.1  4969.5  6864.9  8760.3 10655.7 12551.1  8386.2]]

 [[ 1079.8  2303.1  3683.7  5064.3  6444.9  7825.5  9206.1  6473.4]
  [ 2020.2  4351.8  7019.4  9687.  12354.6 15022.2 17689.8 12435. ]
  [ 2901.6  6313.5 10268.1 14222.7 18177.3 22131.9 26086.5 18339.6]
  [ 3783.   8275.2 13516.8 18758.4 24000.  29241.6 34483.2 24244.2]
  [ 4664.4 10236.9 16765.5 23294.1 29822.7 36351.3 42879.9 30148.8]
  [ 5545.8 12198.6 20014.2 27829.8 35645.4 43461.  51276.6 36053.4]
  [ 6427.2 14160.3 23262.9 32365.5 41468.1 505

In [86]:
print(res[0]-res_ref[0])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
